had to change full disk access to allow terminal 

credit: https://medium.com/@yaskalidis/heres-how-you-can-access-your-entire-imessage-history-on-your-mac-f8878276c6e9

In [74]:
import sqlite3
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
%matplotlib inline
import re

# get contacts dataframe 

In [97]:
address_book_location = "/Users/joycehu/Library/Application Support/AddressBook/Sources/B470B8FA-DC30-4089-8B50-645AD77679C6/AddressBook-v22.abcddb"
conn_add = sqlite3.connect(address_book_location)
cursor = conn_add.cursor()


cursor.execute("SELECT DISTINCT ZABCDRECORD.ZFIRSTNAME [FIRST NAME], ZABCDRECORD.ZLASTNAME [LAST NAME], ZABCDPHONENUMBER.ZFULLNUMBER [FULL NUMBER] FROM ZABCDRECORD LEFT JOIN ZABCDPHONENUMBER ON ZABCDRECORD.Z_PK = ZABCDPHONENUMBER.ZOWNER ORDER BY ZABCDRECORD.ZLASTNAME, ZABCDRECORD.ZFIRSTNAME, ZABCDPHONENUMBER.ZORDERINGINDEX ASC")
result_set = cursor.fetchall()

result_set

contacts = pd.DataFrame(result_set, columns=['first', 'last', 'number'])
contacts

,first,last,number
0,None,None,None
1,None,None,(925) 818-8151
2,None,None,(925) 330-3093
3,None,None,5105851263
4,None,None,(408) 656-8727
...,...,...,...
245,audrey,lee,+14088139475
246,katie,short,(513) 808-3540
247,ellie,wang,+17149320311
248,joanne,yen,+16263227484


In [81]:
contacts.dropna(subset=['number'])

,first,last,number
1,None,None,(925) 818-8151
2,None,None,(925) 330-3093
3,None,None,5105851263
4,None,None,(408) 656-8727
5,None,None,4085544790
...,...,...,...
245,audrey,lee,+14088139475
246,katie,short,(513) 808-3540
247,ellie,wang,+17149320311
248,joanne,yen,+16263227484


In [80]:
contacts['number'].isna().sum()

4

## Clean contacts df

In [91]:
contacts

1                
2                
3                
4                
5                
          ...    
245        audrey
246         katie
247         ellie
248        joanne
249    Tony's Dad
Name: first, Length: 246, dtype: object

In [99]:
'''
cleans numbers and puts them in the same format as numbers in handle df
'''
def standard_numbers(phone_number): 
    # remove all non-numeric characters
    numeric_number = re.sub(r'\D', '', phone_number)
    if len(numeric_number) == 10:
        numeric_number = "+1" + numeric_number
    if len(numeric_number) == 11:
        numeric_number = "+" + numeric_number
    return numeric_number
    
contacts = contacts.dropna(subset=['number'])
contacts['first'] = contacts['first'].fillna('')
contacts['last'] = contacts['last'].fillna('')
contacts['numbers_cleaned'] = contacts['number'].apply(standard_numbers)
contacts['full_name'] = contacts['first'] + ' ' + contacts['last']

contacts

,first,last,number,numbers_cleaned,full_name
1,,,(925) 818-8151,+19258188151,
2,,,(925) 330-3093,+19253303093,
3,,,5105851263,+15105851263,
4,,,(408) 656-8727,+14086568727,
5,,,4085544790,+14085544790,
...,...,...,...,...,...
245,audrey,lee,+14088139475,+14088139475,audrey lee
246,katie,short,(513) 808-3540,+15138083540,katie short
247,ellie,wang,+17149320311,+17149320311,ellie wang
248,joanne,yen,+16263227484,+16263227484,joanne yen


In [67]:
handles[~handles['id'].str.contains('\+1')]['id'].unique()

array(['ballerina.eternity@gmail.com', 'beautyinsight@yahoo.com',
       'oceanhu@yahoo.com', 'gingerhusunny@gmail.com',
       'christopher_z@icloud.com', 'joyce', 'anannya.desh@gmail.com',
       'info@orders.apple.com', 'notserena00@gmail.com', '692632',
       'gasdunertadoxey254@gmail.com', '346637', '78156', '347268',
       '45347', '1113114', '729725', '386732', '242733', '24273',
       'narsingdilawcollege@gmail.com', '46395', '86278',
       'mdabuhanif764479@gmail.com', '25392', '88683', '46339',
       'sinclairsoulia15314@hotmail.com', '26797',
       'sadiasultana04121996@gmail.com', '36246', '89361', '51592',
       '77598', '2865', 'janineee.j9@icloud.com', '83992',
       'hallanthonypwve63798@gmail.com', '1410100004',
       'helenandclara@gmail.com', '53744', '80260',
       'coreentylohggf2397@hotmail.com', '23393', 'ihossaen82@gmail.com',
       '75140', 'moynaakter7722@gmail.com', '1410100005',
       'solfmcfallzm1069@hotmail.com', '83342',
       'crisluttmerde

# get messages dataframe

messages table: contains text messages and relevant info, stores the sender as handle_id
handle table: contains "id" column that contains messager's phone number and corresponding handle_id as "ROWID"

In [3]:
conn = sqlite3.connect('/Users/joycehu/Library/Messages/chat.db')
cur = conn.cursor()

# get handles df
handles = pd.read_sql_query("select * from handle", conn)
handles.head()

,ROWID,id,country,service,uncanonicalized_id,person_centric_id
0,1,+15107378796,US,iMessage,None,c1b59049-da23-476e-aebb-b2e812e48ace
1,2,+15105094802,US,iMessage,None,695bc869-6313-4c1f-b0fe-b9ce9e748742
2,3,+15107095002,US,iMessage,5107095002,0d542d99-cbf5-4254-8c2c-0a929173c7da
3,4,+15107095003,US,iMessage,None,5186b1b7-8016-41db-97d2-130f245f2415
4,5,+16197464242,US,iMessage,None,a5322507-1b30-431d-9ce5-f379526e232a


In [49]:
# get messages

# rowid, text, handle_id, subject, service, date, date_read, date_delivered, is_audio_message, group_title
keep_cols = ['ROWID', 'text', 'handle_id', 'subject', 'service', 'date', 'is_audio_message', 'group_title']

# Construct the SQL query with the specified columns
query = f'''
    SELECT {', '.join(keep_cols)},
           datetime(date/1000000000 + strftime("%s", "2001-01-01"), "unixepoch", "localtime") AS date_utc
    FROM message
'''

messages = pd.read_sql_query(query, conn)
messages

,ROWID,text,handle_id,subject,service,date,is_audio_message,group_title,date_utc
0,1,If haley send cc to my house over the next few...,1,None,iMessage,567238093000000000,0,None,2018-12-22 22:08:13
1,2,Okok got it,1,None,iMessage,567238246000000000,0,None,2018-12-22 22:10:46
2,3,￼￼￼￼,2,None,iMessage,567486800000000000,0,None,2018-12-25 19:13:20
3,4,😉,2,None,iMessage,567487648000000000,0,None,2018-12-25 19:27:28
4,5,https://spice.fsi.stanford.edu/fellowship/chin...,3,None,iMessage,567926776000000000,0,None,2018-12-30 21:26:16
...,...,...,...,...,...,...,...,...,...
174327,175616,None,59,None,iMessage,725498345848999936,0,None,2023-12-28 15:19:05
174328,175617,lol,59,None,iMessage,725500053083445120,0,None,2023-12-28 15:47:33
174329,175618,Didn’t ur mom do art,59,None,iMessage,725500057198278400,0,None,2023-12-28 15:47:37
174330,175619,Same girl?,508,None,iMessage,725500098169796736,0,None,2023-12-28 15:48:18


In [14]:
handles[handles['ROWID'] == 508]

,ROWID,id,country,service,uncanonicalized_id,person_centric_id
505,508,+18582302657,US,iMessage,None,3b3b1944-e0e2-479b-bca4-111a89c639c5


messages columns to keep: 

rowid, text, handle_id, subject, service, date, date_read, date_delivered, is_audio_message, group_title

In [10]:
messages = pd.read_sql_query("select * from message", conn)
messages

,ROWID,guid,text,replace,service_center,handle_id,subject,country,attributedBody,version,...,syndication_ranges,was_delivered_quietly,did_notify_recipient,synced_syndication_ranges,date_retracted,date_edited,was_detonated,part_count,is_stewie,is_kt_verified
0,1,B6E22420-1255-4642-9CAB-F646FC568C00,If haley send cc to my house over the next few...,0,None,1,None,None,b'\x04\x0bstreamtyped\x81\xe8\x03\x84\x01@\x84...,10,...,None,0,0,None,0,0,0,NaN,0,0
1,2,93F3F991-002E-43FB-B128-DA43F5D7ABC7,Okok got it,0,None,1,None,None,b'\x04\x0bstreamtyped\x81\xe8\x03\x84\x01@\x84...,10,...,None,0,0,None,0,0,0,NaN,0,0
2,3,C62EB0E4-F138-4FE2-9220-8E656685DFC6,￼￼￼￼,0,None,2,None,None,b'\x04\x0bstreamtyped\x81\xe8\x03\x84\x01@\x84...,10,...,None,0,0,None,0,0,0,NaN,0,0
3,4,9957C675-F9D8-48B2-AAFD-BC2985E00942,😉,0,None,2,None,None,b'\x04\x0bstreamtyped\x81\xe8\x03\x84\x01@\x84...,10,...,None,0,0,None,0,0,0,NaN,0,0
4,5,D1AE6D88-2933-4820-AD4F-B2B39791A306,https://spice.fsi.stanford.edu/fellowship/chin...,0,None,3,None,None,b'\x04\x0bstreamtyped\x81\xe8\x03\x84\x01@\x84...,10,...,None,0,0,None,0,0,0,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174327,175616,944A0223-1E1F-40E8-A07F-D54CC8DC2049,None,0,None,59,None,None,b'\x04\x0bstreamtyped\x81\xe8\x03\x84\x01@\x84...,10,...,None,0,0,None,0,0,0,1.0,0,0
174328,175617,B449CA77-C1E2-46A8-9F33-6CE1D114666B,lol,0,None,59,None,None,b'\x04\x0bstreamtyped\x81\xe8\x03\x84\x01@\x84...,10,...,None,0,0,None,0,0,0,1.0,0,0
174329,175618,9A556659-9B82-468F-AF85-DD5E95CA6DD5,Didn’t ur mom do art,0,None,59,None,None,b'\x04\x0bstreamtyped\x81\xe8\x03\x84\x01@\x84...,10,...,None,0,0,None,0,0,0,1.0,0,0
174330,175619,177B3446-E0F9-408D-BDE2-EB2DEC3E62A6,Same girl?,0,None,508,None,None,b'\x04\x0bstreamtyped\x81\xe8\x03\x84\x01@\x84...,10,...,None,0,0,None,0,0,0,1.0,0,0


In [9]:
# get the handles to apple-id mapping table
handles = pd.read_sql_query("select * from handle", conn)
# and join to the messages, on handle_id
messages.rename(columns={'ROWID' : 'message_id'}, inplace = True)
handles.rename(columns={'id' : 'phone_number', 'ROWID': 'handle_id'}, inplace = True)
merge_level_1 = temp = pd.merge(messages[['text', 'handle_id', 'date','is_sent', 'message_id']],  handles[['handle_id', 'phone_number']], on ='handle_id', how='left')

In [10]:
# get the chat to message mapping
chat_message_joins = pd.read_sql_query("select * from chat_message_join", conn)
# and join back to the merge_level_1 table
df_messages = pd.merge(merge_level_1, chat_message_joins[['chat_id', 'message_id']], on = 'message_id', how='left')

In [11]:
df_messages

,text,handle_id,date,is_sent,message_id,phone_number,chat_id
0,If haley send cc to my house over the next few...,1,567238093000000000,1,1,+15107378796,1.0
1,Okok got it,1,567238246000000000,0,2,+15107378796,1.0
2,￼￼￼￼,2,567486800000000000,1,3,+15105094802,2.0
3,😉,2,567487648000000000,0,4,+15105094802,2.0
4,https://spice.fsi.stanford.edu/fellowship/chin...,3,567926776000000000,1,5,+15107095002,3.0
5,None,3,568085941000000000,0,6,+15107095002,NaN
6,None,3,568085941000000000,0,7,+15107095002,NaN
7,None,3,568085941000000000,0,8,+15107095002,NaN
8,￼,3,568085941000000000,0,9,+15107095002,4.0
9,Where is that,2,568085996000000000,0,10,+15105094802,4.0


In [6]:
messages['handle_id'].unique()

array([1, 2, 3])

In [35]:
address_book_location = "/Users/joycehu/Library/Application Support/AddressBook/Sources/B470B8FA-DC30-4089-8B50-645AD77679C6/AddressBook-v22.abcddb"
conn_add = sqlite3.connect(address_book_location)
cursor = conn_add.cursor()


cursor.execute("SELECT DISTINCT ZABCDRECORD.ZFIRSTNAME [FIRST NAME], ZABCDRECORD.ZLASTNAME [LAST NAME], ZABCDPHONENUMBER.ZFULLNUMBER [FULL NUMBER] FROM ZABCDRECORD LEFT JOIN ZABCDPHONENUMBER ON ZABCDRECORD.Z_PK = ZABCDPHONENUMBER.ZOWNER ORDER BY ZABCDRECORD.ZLASTNAME, ZABCDRECORD.ZFIRSTNAME, ZABCDPHONENUMBER.ZORDERINGINDEX ASC")
result_set = cursor.fetchall()

result_set

[(None, None, None),
 (None, None, '(925) 818-8151'),
 (None, None, '(925) 330-3093'),
 (None, None, '5105851263'),
 (None, None, '(408) 656-8727'),
 (None, None, '4085544790'),
 (None, None, '5102726600'),
 (None, None, '5106619956'),
 (None, None, '+14254295300'),
 ('AJ', None, '+14025415566'),
 ('Addie', None, '+18582429646'),
 ('Amriel', None, '+16195047072'),
 ('Anais', None, '(510) 455-0703'),
 ('Angela', None, '+14087864910'),
 ('Angie', None, '+1 (805) 895-3815'),
 ('Anna', None, '+19512169482'),
 ('Ashley', None, '+16613199084'),
 ('Ashvi', None, '1 (408) 306-6046'),
 ('Azul', None, '(510) 359-8785'),
 ('Bekah', None, '+15103661021'),
 ('Bianca', None, '+15059107676'),
 ('Bridgette', None, '(510) 322-4309'),
 ('Bryan', None, '+15597879750'),
 ('Canela', None, '+15103292363'),
 ('Carlos', None, '+19252093727'),
 ('Cassidy FMPH', None, '+18183714154'),
 ('Chanyoung', None, '+14085804830'),
 ('Charlotte', None, '+18587059418'),
 ('Christine', None, '(408) 425-1631'),
 ('Coves', N

In [36]:
contacts = pd.DataFrame(result_set, columns=['first', 'last', 'number'])
contacts

,first,last,number
0,None,None,None
1,None,None,(925) 818-8151
2,None,None,(925) 330-3093
3,None,None,5105851263
4,None,None,(408) 656-8727
...,...,...,...
245,audrey,lee,+14088139475
246,katie,short,(513) 808-3540
247,ellie,wang,+17149320311
248,joanne,yen,+16263227484


In [20]:
def get_address_book(address_book_location):
    conn = sqlite3.connect(address_book_location)
    cursor = conn.cursor()

    cursor.execute("SELECT DISTINCT ZABCDRECORD.ZFIRSTNAME [FIRST NAME], ZABCDRECORD.ZLASTNAME [LAST NAME], ZABCDPHONENUMBER.ZFULLNUMBER [FULL NUMBER] FROM ZABCDRECORD LEFT JOIN ZABCDPHONENUMBER ON ZABCDRECORD.Z_PK = ZABCDPHONENUMBER.ZOWNER ORDER BY ZABCDRECORD.ZLASTNAME, ZABCDRECORD.ZFIRSTNAME, ZABCDPHONENUMBER.ZORDERINGINDEX ASC")
    result_set = cursor.fetchall()

    #Convert tuples to json
    json_output = json.dumps([{"FIRSTNAME": t[0], "LASTNAME": t[1], "FULLNUMBER": t[2]} for t in result_set])
    json_list = json.loads(json_output)
    conn.close()

    for obj in json_list:
        # Get the phone number from the object
        phone = obj["FULLNUMBER"]
        if phone is None:
            continue
        # Remove all non-numeric characters from the phone number
        phone = "".join([c for c in phone if c.isnumeric()])
        #if the phone number is 10 digits, add "+1" to the beginning, if it's 11 digits, add "+"
        if len(phone) == 10:
            phone = "+1" + phone
        elif len(phone) == 11:
            phone = "+" + phone
        # Add the phone number to the object
        obj["NUMBERCLEAN"] = phone
        
    new_json_output = json.dumps(json_list)
    return new_json_output

In [ ]:
/Users/joycehu/Library/Messages/chat.db

In [25]:
address_book_location = "/Users/joycehu/Library/Application Support/AddressBook/Sources/B470B8FA-DC30-4089-8B50-645AD77679C6/AddressBook-v22.abcddb"

addressBookData = get_address_book(address_book_location)
addressBookData

'[{"FIRSTNAME": null, "LASTNAME": null, "FULLNUMBER": null}, {"FIRSTNAME": null, "LASTNAME": null, "FULLNUMBER": "(925) 818-8151", "NUMBERCLEAN": "+19258188151"}, {"FIRSTNAME": null, "LASTNAME": null, "FULLNUMBER": "(925) 330-3093", "NUMBERCLEAN": "+19253303093"}, {"FIRSTNAME": null, "LASTNAME": null, "FULLNUMBER": "5105851263", "NUMBERCLEAN": "+15105851263"}, {"FIRSTNAME": null, "LASTNAME": null, "FULLNUMBER": "(408) 656-8727", "NUMBERCLEAN": "+14086568727"}, {"FIRSTNAME": null, "LASTNAME": null, "FULLNUMBER": "4085544790", "NUMBERCLEAN": "+14085544790"}, {"FIRSTNAME": null, "LASTNAME": null, "FULLNUMBER": "5102726600", "NUMBERCLEAN": "+15102726600"}, {"FIRSTNAME": null, "LASTNAME": null, "FULLNUMBER": "5106619956", "NUMBERCLEAN": "+15106619956"}, {"FIRSTNAME": null, "LASTNAME": null, "FULLNUMBER": "+14254295300", "NUMBERCLEAN": "+14254295300"}, {"FIRSTNAME": "AJ", "LASTNAME": null, "FULLNUMBER": "+14025415566", "NUMBERCLEAN": "+14025415566"}, {"FIRSTNAME": "Addie", "LASTNAME": null, 